In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../resources/train.csv', delimiter='|')
df.drop(['id'], axis=1, inplace=True)
df.drop(['job_description'], axis=1, inplace=True)

df_predict = pd.read_csv('../resources/predict_case.csv', delimiter='|')
df_predict.drop(['job_description'], axis=1, inplace=True)

In [5]:
mean_idr_salary = df[df['salary_currency'] == 'IDR']['salary'].mean()
mean_usd_salary = df[df['salary_currency'] == 'USD']['salary'].mean()
mode_experience_level = df['experience_level'].mode()[0]
mode_employment_type = df['employment_type'].mode()[0]
mode_job_benefits = df['job_benefits'].mode()[0]
mode_company_process_time = df['company_process_time'].mode()[0]
mode_company_size = df['company_size'].mode()[0]
mode_company_indusry = df['company_industry'].mode()[0]


# impute 
df.loc[(df['salary_currency'] == 'IDR') & (df['salary'].isnull()), 'salary'] = mean_idr_salary
df.loc[(df['salary_currency'] == 'USD') & (df['salary'].isnull()), 'salary'] = mean_usd_salary
df['salary_currency'] = df['salary_currency'].fillna('IDR')
df['experience_level'] = df['experience_level'].fillna(mode_experience_level)
df['employment_type'] = df['employment_type'].fillna(mode_employment_type)
df['job_benefits'] = df['job_benefits'].fillna(mode_job_benefits)
df['company_process_time'] = df['company_process_time'].fillna(mode_company_process_time)
df['company_size'] = df['company_size'].fillna(mode_company_size)
df['company_industry'] = df['company_industry'].fillna(mode_company_indusry)

df.dtypes

job_title                object
location                 object
salary_currency          object
career_level             object
experience_level         object
education_level          object
employment_type          object
job_function             object
job_benefits             object
company_process_time     object
company_size             object
company_industry         object
salary                  float64
dtype: object

In [4]:
# train model
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train = df[['job_title', 
        'location', 
        'salary_currency', 
        'career_level',
        'experience_level', 
        'education_level', 
        'employment_type',
        'job_function', 
        'job_benefits', 
        'company_process_time', 
        'company_size', 
        'company_industry']]

X_predict = df_predict[['job_title', 
        'location', 
        'salary_currency', 
        'career_level',
        'experience_level', 
        'education_level', 
        'employment_type',
        'job_function', 
        'job_benefits', 
        'company_process_time', 
        'company_size', 
        'company_industry']]

Y_train = df['salary']

X_train = pd.get_dummies(X_train)
X_predict = pd.get_dummies(X_predict)

X_train, X_predict = X_train.align(X_predict, join='left', axis=1, fill_value=0)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

ValueError: Input y contains NaN.

In [55]:
Y_predict = model.predict(X_predict)

In [45]:
# Y_pred = model.predict(X_test)
# mse = mean_squared_error(Y_test, Y_pred)
# accuracy = model.score(X_test, Y_test)
# print(f'Mean Squared Error: {mse}')
# print(f'Accuracy: {accuracy*100}%')

Mean Squared Error: 166945006.6086794
Accuracy: 99.1546671918086%


In [56]:
# write csv id and salary
df_predict['salary'] = Y_predict
mean_idr_salary = df_predict[df_predict['salary_currency'] == 'IDR']['salary'].mean()
mean_usd_salary = df_predict[df_predict['salary_currency'] == 'USD']['salary'].mean()
df_predict.loc[(df_predict['salary_currency'] == 'IDR') & (df_predict['salary'].isnull()), 'salary'] = mean_idr_salary
df_predict.loc[(df_predict['salary_currency'] == 'USD') & (df_predict['salary'].isnull()), 'salary'] = mean_usd_salary
df_predict[['id', 'salary']].to_csv('../resources/predicted_salary.csv', index=False)
